In [1]:
!pip install transformers datasets accelerate peft trl bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 47.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 283.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 196.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 137.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 144.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 126.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 130.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install -U typing_extensions


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 1.7 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
!pip install -U transformers



[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
import transformers
print(transformers.__version__)


4.51.3


In [5]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer,
    BitsAndBytesConfig, EarlyStoppingCallback
)
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import torch

In [6]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import torch
from trl import SFTTrainer
from transformers import EarlyStoppingCallback, TrainingArguments
import typing_extensions

In [7]:
import importlib.metadata
print(importlib.metadata.version("typing_extensions"))


4.13.2


In [8]:
try:
    import importlib.metadata as metadata  # Python 3.8+
except ImportError:
    import importlib_metadata as metadata  # Python 3.7 이하용

packages = ["typing_extensions", "bitsandbytes", "transformers"]

for package in packages:
    try:
        version = metadata.version(package)
        print(f"✅ {package} version: {version}")
    except metadata.PackageNotFoundError:
        print(f"❌ {package} is NOT installed.")


✅ typing_extensions version: 4.13.2
✅ bitsandbytes version: 0.45.5
✅ transformers version: 4.51.3


In [21]:
# 1. CSV 데이터 불러오기
df = pd.read_csv("qa_100_df.csv")

# 2. prompt / response 형식 만들기
def make_prompt(row):
    return f"""질문: {row['question']}
배경 지식: {row['source']}
답변:"""

df["prompt"] = df.apply(make_prompt, axis=1)
df["response"] = df["answer"]

In [22]:
# 3. Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df[["prompt", "response"]])

In [23]:
# 4. Tokenizer & Model 로드 (예: mistral)
model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, token='')
tokenizer.pad_token = tokenizer.eos_token

In [24]:
# 5. 데이터셋에 tokenization 적용 (prompt와 response 모두)
def tokenize_function(examples):
    # 토큰화: prompt는 모델의 입력, response는 모델의 출력
    model_inputs = tokenizer(examples["prompt"], padding="max_length", truncation=True, max_length=512)
    
    # response를 labels로 설정 (모델이 예측해야 할 텍스트)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["response"], padding="max_length", truncation=True, max_length=512)
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 데이터셋에 토크나이즈 적용
tokenized = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [29]:
tokenized = tokenized.rename_columns({"response": "labels"})

In [25]:
# 6. 모델 준비 (QLoRA 구성)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

In [26]:
model = AutoModelForCausalLM.from_pretrained(model_name, token='', quantization_config=bnb_config, device_map="auto")
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [30]:
# 7. 훈련 설정
training_args = TrainingArguments(
    output_dir="./qlora_model_mistral",  # 모델 저장 위치
    per_device_train_batch_size=2,  # 배치 크기
    gradient_accumulation_steps=4,  # 그래디언트 누적
    learning_rate=2e-4,  # 학습률
    num_train_epochs=100,  # 에폭 수
    fp16=True,  # FP16 사용 여부
    logging_dir="./logs",  # 로깅 저장 디렉토리
    logging_steps=5,  # 로깅 주기
    save_steps=12,  # 모델 저장 주기
    save_total_limit=1,  # 저장할 모델 최대 개수
    load_best_model_at_end=True,  # 훈련 종료 시 최고 모델 로드
    metric_for_best_model="loss",  # 최고 모델을 선택할 지표
    greater_is_better=False,  # 더 낮은 값이 더 좋은 경우
    eval_strategy="steps",  # 평가 전략 (steps 또는 epoch)
    save_strategy="steps",  # 저장 전략 (steps 또는 epoch)
    eval_steps=12,  # 평가 주기 (save_steps와 동일하게 설정)
)

In [32]:
# 8. Trainer 실행
trainer = Trainer(
    model=model,  # 훈련할 모델
    args=training_args,  # 훈련 파라미터
    train_dataset=tokenized,  # 훈련 데이터셋
    eval_dataset=tokenized,  # 평가 데이터셋
    tokenizer=tokenizer,  # 모델의 토크나이저
    callbacks=[  # EarlyStopping을 추가
        EarlyStoppingCallback(early_stopping_patience=2)
    ]
)

/tmp/ipykernel_461/4076569658.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
12,10.523500,7.617708


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:1110: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-6824796f-03c9588d2a85ccde33cfb0f1;b6f2cc0f-37b5-4989-85e6-8024a9e18fed)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.1 is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in mistralai/Mistral-7B-Instruct-v0.1.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:236: UserWarning: Could not find a config file in mistralai/Mistral-7B-Instruct-v0.1 - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=Fa

In [ ]:
# 9. 모델 저장
model.save_pretrained("qlora_model_mistral")
tokenizer.save_pretrained("qlora_model_mistral")
